In [1]:
import xgboost
from sklearn.metrics import mean_squared_error
import pandas as pd 
import numpy as np
import Grabber
from Grabber import TSGrabber##This has the Grabber.Frame function, that spits out a time series (hr increment) of avg high price, avg low price, avg high price volume and avg low price volume,
from LatestPriceDatabaseBuilder import Data ##The Dataframe is called Data.Sheet

In [2]:
##This is the database of every item in the game, with real time BuyPrice, SellPrice etc.
Data.Sheet

,ID,BuyPrice,AllTimeHigh,SellPrice,AllTimeLow,Margin,ItemName
0,6,199654,1699111480,182510,1699112406,17144,"""Cannon base"""
1,8,191457,1699111498,182002,1699112410,9455,"""Cannon stand"""
2,10,192987,1699111471,182123,1699112409,10864,"""Cannon barrels"""
3,12,201731,1699111491,191354,1699112419,10377,"""Cannon furnace"""
4,28,324,1699107581,400,1699111483,-76,"""Insect repellent"""
...,...,...,...,...,...,...,...
3733,27785,3204155,1699110553,3050000,1699110212,154155,"""Thammaron's sceptre (au)"""
3734,28146,375000,1699106541,326000,1699112182,49000,"""Log brace"""
3735,28157,35,1699112479,29,1699109645,6,"""Forester's ration"""
3736,28276,88292,1699112435,87000,1699112445,1292,"""Chromium ingot"""


In [3]:
class forecaster():

    def train_test_split(data, cutoffpercent):
        data = data.values 
        n = int(len(data)*(1-cutoffpercent))
        return data[:n] , data[n:]
    
    def IDFinder(item_name):
        item_name =  "\""+ item_name +"\"" 
        df = Data.Sheet
        i = 0
        for x in df['ItemName']:
            if x == item_name: 
                ItemID=(df.iloc[i])['ID']
                break
            else:    
                i += 1
        return ItemID  
    
    def xgb_predict(train,val):
        
        train = np.array(train)
        X, y = train[:,:-1], train[:,-1]
        model = xgboost.XGBRegressor(objective ='reg:squarederror', n_estimators=150)
        model.fit(X,y)
        
        val = np.asarray(val).reshape(1,-1)
        pred = model.predict(val)
        return pred[0]
    
    def validate(data, cutoffpercent):
        predictions = []
        
        train , test = forecaster.train_test_split(data,cutoffpercent)
        history = [x for x in train]
        
        for i in range(len(test)):
            testX, testy = test[i,:-1] , test[i, -1]
            pred = forecaster.xgb_predict(history,testX)
            predictions.append(pred)
            history.append(test[i]) 
        #error = mean_squared_error(test[:,-1], predictions, squared=False)
        #finalpointprediction = xgb_predict(history,test[:,-1])
        return history , test[:,-1][-1]
    def predict(ItemName):
        
        ##Grabs ItemID from ItemName
        ItemID = int(forecaster.IDFinder(ItemName))
        
        ##This section will ultimately forecast the next hour value of the AvgHighPrice
        #################################################################
        ##Creates the required dataframe in a supervised learning format.
        example  = TSGrabber.Frame(ItemID)
        example1 = TSGrabber.Measure(example, 'AvgHighPrice')
        example1 = example1.set_index('Time_hrs')
        example1['t+1'] = example1['AvgHighPrice'].shift(-1)
        example1 = example1.dropna()
        example1['t+1'] = example1['t+1'].astype(int)
        
        ## trains the model with walk forward validation
        x,y  = forecaster.validate(example1, 0.15)
        ##Gets the next hour as a prediction.
        finalpointprediction = forecaster.xgb_predict(x,y)
        growth = finalpointprediction - y 
        #################################################################
        
        ##This section will do the same but for AvgLowPrice
        example  = TSGrabber.Frame(ItemID)
        example1 = TSGrabber.Measure(example, 'AvgLowPrice')
        example1 = example1.set_index('Time_hrs')
        example1['t+1'] = example1['AvgLowPrice'].shift(-1)
        example1 = example1.dropna()
        example1['t+1'] = example1['t+1'].astype(int)
        
        ## trains the model with walk forward validation
        x,y  = forecaster.validate(example1, 0.15)
        ##Gets the next hour as a prediction.
        finalpointprediction1 = forecaster.xgb_predict(x,y)
        growth1 = finalpointprediction1 - y 
    
        
        
        
        print( "In the next hour the average high price of " +ItemName + " will be: " +str(finalpointprediction)+ " which is a hourly change of " +str(growth) +"\n" +
               "In the next hour the average low price of " + ItemName + " will be: " +str(finalpointprediction1)+ " which is a hourly change of " +str(growth1) +"\n"+
               "Thus, the predicted profit margin is " + str(finalpointprediction - finalpointprediction1) )

In [4]:
# This tests the forecaster on some of the games most famous 
forecaster.predict("Rune platebody")
forecaster.predict("Dragon dagger")
forecaster.predict("Abyssal whip")
forecaster.predict("Armadyl godsword")

In the next hour the average high price of Rune platebody will be: 38383.895 which is a hourly change of 109.89453125
In the next hour the average low price of Rune platebody will be: 38337.504 which is a hourly change of 2.50390625
Thus, the predicted profit margin is 46.390625
In the next hour the average high price of Dragon dagger will be: 17357.008 which is a hourly change of 37.0078125
In the next hour the average low price of Dragon dagger will be: 17310.074 which is a hourly change of 51.07421875
Thus, the predicted profit margin is 46.933594
In the next hour the average high price of Abyssal whip will be: 1374937.9 which is a hourly change of -9340.125
In the next hour the average low price of Abyssal whip will be: 1360111.4 which is a hourly change of -8995.625
Thus, the predicted profit margin is 14826.5
In the next hour the average high price of Armadyl godsword will be: 11011449.0 which is a hourly change of 10723.0
In the next hour the average low price of Armadyl godswor